In [12]:
import cryptocompare
import pandas as pd
import datetime

import datetime
import requests
import json
from functools import reduce
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


CRYPTO_COMPARE_KEY = "6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4"
cryptocompare.cryptocompare._set_api_key_parameter(CRYPTO_COMPARE_KEY)

'&api_key=6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4'

In [2]:
def get_historical_data(coin:str,
                        cur:str,
                        ts=int(datetime.datetime.now().timestamp()),
                        limit=2000):
    data = []
    done = False
    while not done:
        try:
            tmp_json = cryptocompare.get_historical_price_hour(coin, cur, limit=limit, toTs=ts)
            if tmp_json[0]['volumefrom'] == 0 and tmp_json[0]['volumeto'] == 0:
                done = True
            else:
                data.extend(tmp_json)
                ts -= 3600000
                done = False
        except Exception as err:
            print(err)
            done = True
    return data

In [3]:
list_of_coins = ['BTC', 'ETH', 'BUSD', 'USDT']
list_of_currencies = ['USD', 'JPY', 'BTC']

In [9]:
import json
import boto3
from botocore.exceptions import ClientError
import datetime


def upload_json_s3(data, bucket: str, path: str, s3_client=None):
    """
    This function is used to upload a raw json data to an S3 bucket
    Params
        data: some data to upload to S3
        bucket: name of the bucket
        path: path for file within the bucket
        s3_client: client for S3
    Return
        True if tables were created, otherwise False 
    """
    if not s3_client:
        s3_client = boto3.client('s3')
    try:
        s3_client.put_object(Body=json.dumps(data), Bucket=bucket, Key=path)        
    except ClientError as e:
        print(e)
        return False
    return True

In [5]:
AWS_BUCKET = 'kosmobiker-masterproject'
session = boto3.Session(profile_name='master')
s3_client = session.client('s3')

for coin in list_of_coins:
    for cur in list_of_currencies:
        if coin != cur:
            try:
                data = get_historical_data(coin, cur)
                filename = f"{coin}-{cur}"
                path = f"data/test/input/{filename}.json"
                upload_json_s3(data, AWS_BUCKET, path, s3_client)
                print(f"{filename} was uploaded!")
            except Exception as err:
                print(err)

BTC-USD was uploaded!
BTC-JPY was uploaded!
ETH-USD was uploaded!
ETH-JPY was uploaded!
ETH-BTC was uploaded!
BUSD-USD was uploaded!
BUSD-JPY was uploaded!
BUSD-BTC was uploaded!
USDT-USD was uploaded!
USDT-JPY was uploaded!
USDT-BTC was uploaded!


In [10]:
def get_daily_data(coin:str,
                    cur:str,
                    toTs=datetime.datetime.now()) -> List[Dict]:
    try:
        return cryptocompare.get_historical_price_minute(coin, cur)
    except Exception as err:
        print(err)


In [14]:
type(get_daily_data('ETH', 'USDT'))

list